In [2]:
#copy file from trained model
!mkdir model
#copy model file
!cp -r ../outputs/tasd/shulex/extraction/cktepoch=1.ckpt ./model/

mkdir: cannot create directory ‘model’: File exists


In [3]:
!cat Dockerfile

ARG REGISTRY_URI
FROM ${REGISTRY_URI}/pytorch-inference:1.5.0-gpu-py36-cu101-ubuntu16.04

RUN mkdir -p /opt/ml/model

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

##########################################################################################
# SageMaker requirements
##########################################################################################
## install flask
RUN pip install networkx==2.3 flask gevent gunicorn boto3 transformers==4.6.0 datasets==1.11.0 sentencepiece==0.1.91 pytorch_lightning==0.8.1 jieba editdistance -i https://opentuna.cn/pypi/web/simple
### Install nginx notebook
RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# forward request and error logs to docker log collector
RUN ln -sf /dev/stdout /var/log/nginx/access.log
RUN ln -sf /dev/stderr /var/log/nginx/error.log

# Set up the progr

In [5]:
#terminal run faster
##清理缓存 sagemaker
# sudo vim /etc/docker/daemon.json
# { “data-root”: “/home/ec2-user/SageMaker/docker” }
# sudo service docker restart
!sh build_and_push.sh

set -e
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
image=$1

if [ "$image" == "" ]
then
    echo "Use image name absa"
    image="absa"
fi
Use image name absa

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi

# Get the region defined in the current configuration
region=$(aws configure get region)
#regions=$(aws ec2 describe-regions --all-regions --query "Regions[].{Name:RegionName}" --output text)

#for region in $regions; do

#aws s3 cp s3://aws-solutions-${region}/spot-bot-models/cars/model.tar.gz ./
#tar zxvf model.tar.gz
# TODO: update regional location based on https://amazonaws-china.com/releasenotes/available-deep-

In [14]:
#注意修改：847380964353.dkr.ecr.ap-northeast-1.amazonaws.com/generative-absa为自己对应的

!python create_endpoint.py \
--endpoint_ecr_image_path "847380964353.dkr.ecr.us-west-2.amazonaws.com/absa" \
--endpoint_name 'generative-absa-v3' \
--instance_type "ml.p3.2xlarge"

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
model_name:  generative-absa-v3
endpoint_ecr_image_path:  847380964353.dkr.ecr.us-west-2.amazonaws.com/absa
<<< Completed model endpoint deployment. generative-absa-v3


In [1]:
%%time 

#创建约需要5-10min,等到显示inservice可以使用
from boto3.session import Session
import json
data={"data": 'very high quality and sturdy Very high quality. We love this and intend to buy more. My 15 year old took mine!'}
session = Session()
    
runtime = session.client("runtime.sagemaker")
response = runtime.invoke_endpoint(
    EndpointName='generative-absa-v3',
    ContentType="application/json",
    Body=json.dumps(data),
)

result = json.loads(response["Body"].read())
print (result)

{'result': '(will buy more, will_repurchase)', 'infer_time': '0:00:00.590412'}
CPU times: user 197 ms, sys: 20.5 ms, total: 218 ms
Wall time: 921 ms


In [15]:
import sagemaker as sage

endpointName='generative-absa'
endpoint_config_name = endpointName + '-config'
model_name = endpointName
sage.delete_endpoint(EndpointName = endpointName)
sage.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sage.delete_model(ModelName=model_name)

AttributeError: module 'sagemaker' has no attribute 'delete_endpoint'